In [126]:
phrase_path = '/home/ayhaos/rebuttal/COAT/phrases.py'
phrases = {
    "heating(source)": "an object that can act as a heating source to warm/cook food",
    "carrying": "an object that can be used to carry liquid or solid from one place to another",
    "cleaning": "an object that can be used for cleaning or wiping something",
    "washing": "an object where we can wash something",
    "cutting": "an object that can be used to cut something",
    "disposing": "an object in which we can throw waste water or garbage",
    "mixing(tool)": "an object that can be used to mix and  blend something in a bowl",
    "mixing(vessel)": "a vessel that can be used to mix and blend something using a spoon",
    "heating(vessel)": "an vessel that can be used to heat food on some heating source like microwave or stove",
    "storage": "an object that can be used to store other physical items",
    "entertainment": "an object to watch movies, or play digital games",
    "comfort": "an object that allows human to rest over it",
    "reading": "an object that can be used to read stories/news while sitting on a chair",
    "increasing_height": " an object using which we can reach high places in house and increase our height",
    "time": "an object using which we can infer exact time",
    "eating": "an edible object",
    "physical_excercise": "an object that can be used for physical excercise",
    "writing": "an object that can be used to write or document ideas",
    "surface_support": "an object which can provide a resting support to keep stuff like laptop or sit on it",
    "light_source": "an object that can alter the brightness of a room",
    "decoration": "an obejct that can be used as a decoration",
    "breaking": "an object that can be used to break eggs"
}

In [127]:
import pdb, json

In [128]:
def create_mistral_prompt(question, options):
    new_prompt = question + "\nYou should focus on safety, feasibility, minimum effort, minimum time aspects.\n" + options + "\nChoose the option that would be best. Your answer should have the object name. Do not give any reason. Answer should be a JSON Dictionary with key as keyword 'answer' and the value should be your chosen option. Example Answer : \{\"answer\" : \"StoveBurner\"\}"
    
    new_prompt = f"""<s>[INST]{new_prompt}[/INST] I'm an intelligent household agent, I can help you with deciding what object to use for a task</s>[INST]Suggest me the correct option in JSON format "id": object option ID?[/INST]"""

    return new_prompt


def create_palm_prompt(question, options, task):

    if task in [2]:
            keys_to_remove = ['material_penalty', 'time_penalty']
            # pdb.set_trace()
            # Using list comprehension to create a new list of dictionaries with specified keys removed
            options_list = [{outer_key: {inner_key: inner_value for inner_key, inner_value in inner_dict.items() if inner_key not in keys_to_remove} for outer_key, inner_dict in outer_dict.items()} for outer_dict in options]
    elif task in [1]:
         options_list = options
    if task in ['u',0]:
        prompt = str(question) +'\n' + str(options) + "\nChoose the option that would be best. Answer should be a JSON Dictionary. Do not give any reason. For example a sample answer looks like: {\"answer\": \"<your chosen option's alphabet code between [A/B/C/D/E]>\"}"
 
    elif task in [1,2]:
        prompt = str(question) +'\n' + "Here, \'already_in_use\' variable depicts object's availability. \'reversible_using\' means an object is temporarily busy but can be used after some waiting time but not immediately.\n For example: could be wet, is getting charged, is being used by someone else etc,  Whereas, irreversible_using means that an object is depleted and cannot be used." + '\n\n' + str(options_list) + "\nChoose the object that would be best considering all the physical variables.You should focus on safety, fragility of object, material suitablity and minimum time aspects. Hint: cleaning takes lesser time than waiting for an object to become available. Do not give any reason. Answer should be a JSON Dictionary. For example a sample answer looks like: {\"answer\": \"<chosen option [A/B/C/D/E]>\"}"

    return prompt


def create_palm_format_dataset(question, option, task, answer=None):
    if answer is not None:
        return {
            "input_text": create_palm_prompt(question, option, task),
            "output_text": str({'answer': answer})
        }
    pdb.set_trace()
    # return {
    #     "input_text": create_palm_prompt(question, option),
    #     "output_text": ""
    # }

In [129]:
# create_palm_format_dataset(question, option, task, answer)

In [130]:
import json
from pprint import pprint

# gt_ad = {'u':'/home/ayhaos/rebuttal/COAT/ft_data/task_u/Variation_5/GT_used/raw_question_answer_database.json',
#          0: '/home/ayhaos/rebuttal/COAT/ft_data/task_0/Variation_4/GT_used/raw_question_answer_database.json',
#          1: '/home/ayhaos/rebuttal/COAT/ft_data/task_1/Variation_1_2_3/GT_used/raw_question_answer_database.json',
#          2: '/home/ayhaos/rebuttal/COAT/ft_data/task_2/Variation_1_2_3_4_5/GT_used/raw_question_answer_database.json'}
# eval_gt = {'u':'/home/ayhaos/rebuttal/COAT/ft_data/task_u/Variation_5/eval/raw_question_answer_database.json',
#          0: '/home/ayhaos/rebuttal/COAT/ft_data/task_0/Variation_4/eval/raw_question_answer_database.json',
#          1: '/home/ayhaos/rebuttal/COAT/ft_data/task_1/Variation_1_2_3/eval/raw_question_answer_database.json',
#          2: '/home/ayhaos/rebuttal/COAT/ft_data/task_2/Variation_1_2_3_4_5/eval/raw_question_answer_database.json'}

# with open(f'/home/ayhaos/rebuttal/COAT/ft_data/task_u/Variation_5/eval/raw_question_answer_database.json', 'r') as json_file:
#     data = json.load(json_file)
#     print('total data:', len(data))

# with open('/home/ayhaos/rebuttal/COAT/ft_data/task_u_0_combined/eval/processed_question_answer_database.jsonl', "w") as jsonl_file:
#     for sample in data:
#         question = sample['question']
#         option = sample['options']
#         answer = sample['correct_answer']

#         json_obj = create_palm_format_dataset(question, option, 'u',answer)

#         # Write to JSONL file
#         jsonl_file.write(json.dumps(json_obj))
#         jsonl_file.write("\n")

#     with open(f'/home/ayhaos/rebuttal/COAT/ft_data/task_0/Variation_4/eval/raw_question_answer_database.json', 'r') as json_file:
#         data = json.load(json_file)
#         print('total data:', len(data))
#     for sample in data:
#         question = sample['question']
#         option = sample['options']
#         answer = sample['correct_answer']

#         json_obj = create_palm_format_dataset(question, option,0, answer)

#         # Write to JSONL file
#         jsonl_file.write(json.dumps(json_obj))
#         jsonl_file.write("\n")

    

In [131]:
# create_mistral_prompt(question, option)

In [132]:
with open(f'/home/ayhaos/rebuttal/COAT/ft_data/task_1/Variation_1_2_3/GT_used/raw_question_answer_database.json', 'r') as json_file:
    data = json.load(json_file)
    print('total data:', len(data))

with open('/home/ayhaos/rebuttal/COAT/ft_data/task_1_2_combined/GT_used/processed_question_answer_database.jsonl', "w") as jsonl_file:
    for sample in data:
        question = sample['question']
        option = sample['options']
        answer = sample['correct_answer']
        utility = sample['context']
        
        # pdb.set_trace()
        json_obj = create_palm_format_dataset(question.replace(utility,phrases[utility],1), option,1,answer)
        # pdb.set_trace()
        # Write to JSONL file
        jsonl_file.write(json.dumps(json_obj))
        jsonl_file.write("\n")

    with open(f'/home/ayhaos/rebuttal/COAT/ft_data/task_2/Variation_1_2_3_4_5/GT_used/raw_question_answer_database.json', 'r') as json_file:
        data = json.load(json_file)
        print('total data:', len(data))
    for sample in data:
        question = sample['question']
        option = [sample['options']]
        answer = sample['correct_answer']

        json_obj = create_palm_format_dataset(question.replace(utility,phrases[utility],1), option,2, answer)

        # Write to JSONL file
        jsonl_file.write(json.dumps(json_obj))
        jsonl_file.write("\n")


with open(f'/home/ayhaos/rebuttal/COAT/ft_data/task_1/Variation_1_2_3/eval/raw_question_answer_database.json', 'r') as json_file:
    data = json.load(json_file)
    print('total data:', len(data))

with open('/home/ayhaos/rebuttal/COAT/ft_data/task_1_2_combined/eval/processed_question_answer_database.jsonl', "w") as jsonl_file:
    for sample in data:
        question = sample['question']
        option = sample['options']
        answer = sample['correct_answer']
        utility = sample['context']
        
        # pdb.set_trace()
        json_obj = create_palm_format_dataset(question.replace(utility,phrases[utility],1), option,1,answer)
        # pdb.set_trace()
        # Write to JSONL file
        jsonl_file.write(json.dumps(json_obj))
        jsonl_file.write("\n")

    with open(f'/home/ayhaos/rebuttal/COAT/ft_data/task_2/Variation_1_2_3_4_5/eval/raw_question_answer_database.json', 'r') as json_file:
        data = json.load(json_file)
        print('total data:', len(data))
    for sample in data:
        question = sample['question']
        option = [sample['options']]
        answer = sample['correct_answer']

        json_obj = create_palm_format_dataset(question.replace(utility,phrases[utility],1), option,2, answer)

        # Write to JSONL file
        jsonl_file.write(json.dumps(json_obj))
        jsonl_file.write("\n")


total data: 450
total data: 750
total data: 416
total data: 706


In [135]:
#Truncating to 250 examples for eval
with open('/home/ayhaos/rebuttal/COAT/ft_data/task_1_2_combined/eval/processed_question_answer_database.jsonl', 'r') as infile, open('/home/ayhaos/rebuttal/COAT/ft_data/task_1_2_combined/eval/truncated_250_processed_question_answer_database.jsonl.jsonl', 'w') as outfile:
    for index, line in enumerate(infile):
        # Check if the line number is not a multiple of 4
        if (index + 1) % 5 == 0:
            # Write the line to the output file
            outfile.write(line)
            
with open('/home/ayhaos/rebuttal/COAT/ft_data/task_u_0_combined/eval/processed_question_answer_database.jsonl', 'r') as infile, open('/home/ayhaos/rebuttal/COAT/ft_data/task_u_0_combined/eval/truncated_200_processed_question_answer_database.jsonl.jsonl', 'w') as outfile:
    for index, line in enumerate(infile):
        # Check if the line number is not a multiple of 4
        if (index + 1) % 2 == 0:
            # Write the line to the output file
            outfile.write(line)
            
